In [18]:
from overcooked_ai_py.mdp.overcooked_env import OvercookedEnv, Overcooked
from overcooked_ai_py.mdp.overcooked_mdp import OvercookedGridworld

base_mdp = OvercookedGridworld.from_layout_name("cramped_room") # or other layout
assert isinstance(base_mdp, OvercookedGridworld)
base_env = OvercookedEnv.from_mdp(mdp=base_mdp, info_level=0, horizon=400)
env = Overcooked(base_env=base_env, featurize_fn=base_env.featurize_state_mdp)

env.reset()

for _ in range(100):

    print(base_mdp.get_actions(env.reset()))
    result = env.step()
    print(result)







AttributeError: 'dict' object has no attribute 'objects'